In [2]:
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt

import jax_cfd.base as cfd
import jax_cfd.base.grids as grids
import jax_cfd.spectral as spectral
from jax_cfd.spectral import utils as spectral_utils

from jax import vmap, jit
# 导入 jax 库，用于高性能的数值计算和自动求导
# 导入 jax.numpy 作为 jnp，它提供了与 numpy 类似的接口，用于在 jax 中进行数值运算
# 导入 matplotlib.pyplot 用于数据可视化
# 导入 jax_cfd 库的基础模块，用于计算流体动力学的基础操作
# 导入 jax_cfd 库的网格模块，用于定义和操作网格
# 导入 jax_cfd 库的谱方法模块，用于基于谱方法的流体模拟
# 导入 jax_cfd 谱方法的工具模块，提供谱方法相关的实用工具
# 导入 jax 的 vmap 和 jit 函数，vmap 用于自动向量化操作，jit 用于即时编译以提高性能

物理参数设置与模拟准备

In [7]:
%%time  # 此命令用于测量代码块的执行时间


# 物理参数
viscosity = 1e-2 # 粘性系数
max_velocity = 3 # 最大速度

# 创建一个二维网格，网格大小为 1024x1024，定义域为 [0, 2π] x [0, 2π]
grid = grids.Grid((1024, 1024), domain=((0, 2 * jnp.pi), (0, 2 * jnp.pi))) 

# 根据稳定时间步长公式计算时间步长
# dt = cfd.equations.stable_time_step(max_velocity, .5, viscosity, grid)  

# 手动设置时间步长为 5e-4
dt = 5e-4


# setup step function using crank-nicolson runge-kutta order 4
smooth = True # use anti-aliasing 
step_fn = spectral.time_stepping.crank_nicolson_rk4(
    spectral.equations.NavierStokes2D(viscosity, grid, smooth=smooth), dt)
# 使用 Crank-Nicolson 方法结合四阶 Runge-Kutta 方法来设置时间步长函数
# 启用抗锯齿功能，用于减少模拟中的高频噪声
# 创建一个时间步长函数，用于求解二维 Navier-Stokes 方程

# run the simulation up until time 25.0 but only save 10 frames for visualization
# final_time = 10.0
outer_steps = 201
inner_steps = 100
# 模拟运行的总步数，这里设置为 201 步
# 每多少步进行一次保存，这里设置为 100 步

trajectory_fn = cfd.funcutils.trajectory(
    cfd.funcutils.repeated(step_fn, inner_steps), outer_steps)
# 创建一个轨迹函数，用于生成模拟的轨迹
# 该函数会重复执行 step_fn 指定的次数（inner_steps），并重复 outer_steps 次

UsageError: Can't use statement directly after '%%time'!


初始化速度场并进行模拟

In [5]:
w = jnp.fft.irfftn(trajectory, axes=(1,2))

velocity_solve = spectral_utils.vorticity_to_velocity(grid)

u_hat, v_hat = vmap(velocity_solve)(trajectory)
u = vmap(jnp.fft.irfftn)(u_hat)
v = vmap(jnp.fft.irfftn)(v_hat)

x = jnp.arange(grid.shape[0]) * 2 * jnp.pi / grid.shape[0]
y = jnp.arange(grid.shape[0]) * 2 * jnp.pi / grid.shape[0]
t = dt * jnp.arange(outer_steps) * inner_steps

u0 = u[0, :, :]
v0 = v[0, :, :]
w0 = w[0, :, :]

data = {'w': w, 'u':u, 'v':v, 'u0':u0, 'v0':v0, 'w0':w0, 'x':x, 'y': y, 't':t, 'viscosity':viscosity}
# jnp.save('ns_tori_high_res.npy', data)

NameError: name 'trajectory' is not defined

In [4]:
# Downsampling
res = 8
data = {'u': u[:, ::res, ::res], 
        'v': v[:, ::res, ::res], 
        'w': w[:, ::res, ::res], 
        'x': x[::res], 
        'y': y[::res], 
        't':t, 
        'viscosity':viscosity}

jnp.save('ns_tori.npy', data)